In [1]:
import os
import sys
import pandas as pd 
import psycopg2



In [2]:
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0,rpath)
from src import utils

In [3]:
import sys
sys.path.insert(0, '../db_connection/')
from db_connection.database import Database

In [4]:
conn = Database(host='localhost',port=5432,user='basilel',dbname='telecom',password='Basi')

In [5]:
conn.connect()


In [16]:
#Top 10 handset used by the customers
query= 'SELECT "Handset Type", COUNT(*) AS column_count FROM xdr_data GROUP BY "Handset Type" ORDER BY column_count DESC LIMIT 10;' 
result = conn.execute_query(query=query)
print(utils.print_table(result,["HandSet Type", "User Count"]))

+------------------------------+------------+
|         HandSet Type         | User Count |
+------------------------------+------------+
|       Huawei B528S-23A       |   19752    |
|   Apple iPhone 6S (A1688)    |    9419    |
|    Apple iPhone 6 (A1586)    |    9023    |
|          undefined           |    8987    |
|    Apple iPhone 7 (A1778)    |    6326    |
|   Apple iPhone Se (A1723)    |    5187    |
|    Apple iPhone 8 (A1905)    |    4993    |
|   Apple iPhone Xr (A2105)    |    4568    |
| Samsung Galaxy S8 (Sm-G950F) |    4520    |
|    Apple iPhone X (A1901)    |    3813    |
+------------------------------+------------+


In [15]:
#Top 3 handset manufacturer
query1= 'SELECT "Handset Manufacturer",COUNT(*) AS column_count FROM xdr_data GROUP BY "Handset Manufacturer" ORDER BY COUNT(*) DESC LIMIT 3;' 
result = conn.execute_query(query=query1)
print(utils.print_table(result,["HandSet Manufacturer" ,"User Count"]))

+----------------------+------------+
| HandSet Manufacturer | User Count |
+----------------------+------------+
|        Apple         |   59565    |
|       Samsung        |   40839    |
|        Huawei        |   34423    |
+----------------------+------------+


In [8]:
# top 5 handsets per top 3 handset manufacturer
query2= '''WITH RankedManufacturers AS (
  SELECT
    "Handset Manufacturer",
    COUNT(*) AS manufacturer_count,
    ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS manufacturer_rank
  FROM xdr_data
  GROUP BY "Handset Manufacturer"
),
RankedHandsets AS (
  SELECT
    "Handset Type",
    "Handset Manufacturer",
    COUNT(*) AS handset_count,
    ROW_NUMBER() OVER (PARTITION BY "Handset Manufacturer" ORDER BY COUNT(*) DESC) AS handset_rank
  FROM xdr_data
  GROUP BY "Handset Manufacturer", "Handset Type"
)
SELECT
  rh."Handset Manufacturer",
  rh."Handset Type",
  rh.handset_count
FROM RankedHandsets rh
JOIN RankedManufacturers rm ON rh."Handset Manufacturer" = rm."Handset Manufacturer"
WHERE rh.handset_rank <= 5 AND rm.manufacturer_rank <= 3
''' 
result1 = conn.execute_query(query=query2)
print(utils.print_table(result1,["HandSet Type","HandSet Manufacturer" ,"User Count"]))

+--------------+--------------------------------+------------+
| HandSet Type |      HandSet Manufacturer      | User Count |
+--------------+--------------------------------+------------+
|    Apple     |    Apple iPhone 6S (A1688)     |    9419    |
|    Apple     |     Apple iPhone 6 (A1586)     |    9023    |
|    Apple     |     Apple iPhone 7 (A1778)     |    6326    |
|    Apple     |    Apple iPhone Se (A1723)     |    5187    |
|    Apple     |     Apple iPhone 8 (A1905)     |    4993    |
|    Huawei    |        Huawei B528S-23A        |   19752    |
|    Huawei    |          Huawei E5180          |    2079    |
|    Huawei    | Huawei P20 Lite Huawei Nova 3E |    2021    |
|    Huawei    |           Huawei P20           |    1480    |
|    Huawei    |         Huawei Y6 2018         |    997     |
|   Samsung    |  Samsung Galaxy S8 (Sm-G950F)  |    4520    |
|   Samsung    |   Samsung Galaxy A5 Sm-A520F   |    3724    |
|   Samsung    |  Samsung Galaxy J5 (Sm-J530)   |    36

In [9]:
query = "SELECT * FROM xdr_data;"
df = pd.read_sql(query, conn.connection)
df['Bearer Id'].duplicated().any()

/var/folders/04/3b1dms951y10x8q5rqfj4twr0000gn/T/ipykernel_13576/1302470016.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn.connection)


True

In [10]:
# Group by User and calculate the number of sessions
xDR_sess_agg = df['Bearer Id'].value_counts()
xDR_sess_agg.to_frame()

,count
Bearer Id,
1.304243e+19,16
1.304243e+19,10
1.304243e+19,9
1.304243e+19,9
1.304243e+19,9
...,...
1.311448e+19,1
1.311448e+19,1
7.277826e+18,1


In [11]:
#Group by User and calculate the duration of each session
xDR_duration_agg = df.groupby('Bearer Id')['Dur. (ms)'].sum()
xDR_duration_agg.to_frame()

,Dur. (ms)
Bearer Id,
6.917538e+18,24534.0
6.917538e+18,21489.0
6.917538e+18,27786.0
6.917538e+18,15635.0
6.917538e+18,24264.0
...,...
1.318654e+19,80024.0
1.318654e+19,145291.0
1.318654e+19,86399.0


In [12]:
#Group by User and calculate the total upload (Bytes) during each session
xDR_upload_agg = df.groupby('Bearer Id')['Total UL (Bytes)'].sum()
xDR_upload_agg.to_frame()

,Total UL (Bytes)
Bearer Id,
6.917538e+18,41704610.0
6.917538e+18,38693596.0
6.917538e+18,15548926.0
6.917538e+18,49605688.0
6.917538e+18,21983463.0
...,...
1.318654e+19,35412664.0
1.318654e+19,36189587.0
1.318654e+19,35919460.0


In [13]:
#Group by User and calculate the total download (Bytes) during each session
xDR_download_agg = df.groupby('Bearer Id')['Total DL (Bytes)'].sum()
xDR_download_agg.to_frame()

,Total DL (Bytes)
Bearer Id,
6.917538e+18,500721999.0
6.917538e+18,709549265.0
6.917538e+18,854465860.0
6.917538e+18,825786510.0
6.917538e+18,624957763.0
...,...
1.318654e+19,145293140.0
1.318654e+19,387001634.0
1.318654e+19,872688792.0


In [17]:
xDR_allapp_agg = df.groupby('Bearer Id').agg({'Youtube DL (Bytes)':sum,'Youtube UL (Bytes)':sum,'Netflix DL (Bytes)':sum,'Netflix UL (Bytes)':sum,'Gaming DL (Bytes)':sum,'Gaming UL (Bytes)':sum,'Other DL (Bytes)':sum,'Other UL (Bytes)':sum})
xDR_allapp_agg

,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes)
Bearer Id,,,,,,,,
6.917538e+18,6139644.0,2071526.0,19494278.0,14668354.0,466109357.0,5333340.0,670751043.0,15950724.0
6.917538e+18,10281221.0,18119976.0,19455048.0,10631652.0,673282567.0,2670856.0,501608458.0,3908870.0
6.917538e+18,5385159.0,4295851.0,15755839.0,1300571.0,821879090.0,8521398.0,472846860.0,1337849.0
6.917538e+18,12347020.0,11089528.0,2859358.0,1738176.0,805301713.0,16257481.0,24303797.0,15907613.0
6.917538e+18,15231815.0,8401567.0,21563985.0,2817981.0,583864716.0,6992868.0,685122214.0,2966860.0
...,...,...,...,...,...,...,...,...
1.318654e+19,12404964.0,4343114.0,11108134.0,21649273.0,114093049.0,2834548.0,695881178.0,3888729.0
1.318654e+19,22147919.0,2152449.0,21468525.0,8603105.0,328766801.0,7569327.0,371261255.0,12947410.0
1.318654e+19,11879062.0,1290963.0,22596930.0,11943452.0,833634251.0,10607174.0,697260277.0,9094407.0
